# Python整理資料實務2 — 利用正規表示式一次轉換多項欄位資料

此篇為Python整理資料實務第二篇 ([文章](https://peisyuanli.medium.com/python%E6%95%B4%E7%90%86%E8%B3%87%E6%96%99%E5%AF%A6%E5%8B%992-%E5%88%A9%E7%94%A8%E6%AD%A3%E8%A6%8F%E8%A1%A8%E7%A4%BA%E5%BC%8F%E4%B8%80%E6%AC%A1%E8%BD%89%E6%8F%9B%E5%A4%9A%E9%A0%85%E6%AC%84%E4%BD%8D%E8%B3%87%E6%96%99-3eb1c762b963))使用的程式碼。

實務資料整理中，常需要轉換欄位資料，但如何用有效率的方式一次轉換需要轉換的欄位? 本篇文章藉由正規表示式 (Regular Expression) 一次轉換多項欄位資料。實務上遇到的資料欄位可能更複雜或不一定有固定的pattern，以簡化的範例來說明正規表示式的應用，並找出相同pattern的欄位來進行轉換。

## 情境和資料

假設要處理產線某產品的測試資料，每隔10分鐘紀錄一次，紀錄項目包含產品中各模組的電流、溫度、風扇負載率等。範例資料如下，其中數值為非實際數值。

需要進行清理資料將部分欄位的資料變成可理解的數值/文字，如：電流和溫度紀錄為四位數，須轉換成各自常見單位對應的數值。欲進行轉換的項目如下：

1. 兩種電流 (Current、Sensor_current) 要轉換成一位數的數值 (單位為安培)，將原先數值除以1000。
2. 溫度 (Temperature) 要轉換成兩位數的數值 (單位為度)，將原先數值除以100。
3. 風扇負載率 (Fan_duty) 紀錄為兩位數，將原先數值除以100後則為使用的比例。
4. 風扇負載率警示 (Fan_duty_alert) 紀錄為1和0，代表風扇負載率是否超過80%，分別轉為”Yes” 和 “No”。

### Step 1: 讀資料

In [1]:
import pandas as pd
file = 'data_example.xlsx'
df = pd.read_excel( file )

In [2]:
df

,Serial_number,Record_time,Current_1,Current_2,Sensor_current_1,Sensor_current_2,Temperature_1,Temperature_2,Fan_duty_1,Fan_duty_2,Fan_duty_alert_1,Fan_duty_alert_2
0,A503,2021-05-24 09:00:06,3100,3046,3028,3100,2970,2780,60,74,0,0
1,A503,2021-05-24 09:10:06,3716,3662,3644,3734,4350,4188,60,74,0,0
2,A503,2021-05-24 09:20:06,3716,3662,3644,3734,4350,4188,58,72,0,0
3,A503,2021-05-24 09:30:07,3189,3136,3118,3189,3212,3034,56,70,0,0
4,A503,2021-05-24 09:40:06,3878,3824,3788,3878,4588,4472,56,70,0,0
5,A503,2021-05-24 09:50:07,3878,3824,3788,3878,4588,4472,58,72,0,0
6,A503,2021-05-24 10:00:06,3118,3064,3046,3118,2910,2750,74,70,0,0
7,A503,2021-05-24 10:10:06,3752,3716,3680,3770,4513,4300,77,70,0,0
8,A503,2021-05-24 10:20:07,3509,3678,3555,3670,4430,4230,82,71,1,0
9,A503,2021-05-24 10:30:06,3207,3153,3136,3207,2812,2656,84,72,1,0


### Step 2: 設定取欄位的pattern

利用迴圈和正規表示式一次轉換多項欄位資料，設定欲轉換欄位的pattern，搭配 str.match method 找出符合pattern的欄位項目，再一次進行轉換，各項正規表示式表示如下：

正規表示式的pattern視要處理的文字或欄位名稱來進行調整，也可以先將所有欄位改成大寫，再定義pattern。正規表示式的相關定義請見參考網址。

**1. 兩種電流： ‘.*[Cc]urrent’**

符合的欄位為Current_1、Current_2、Sensor_current_1和Sensor_current_2，將原先數值除以1000。
- “c”有大寫和小寫, 使用[Cc]
- “Sensor_current_1”中”current”之前有多個字元，”Current_1"中”Current”前沒有字元，使用 “.*”，其中 “.” 代表任意字元，”*” 代表0個或多個字元

**2. 溫度：’Temperature’**

符合的欄位為Temperature_1和Temperature_2，將原先數值除以100。

**3.風扇負載率：’.+(duty_)[1–9]’**

符合的欄位為Fan_duty_1和Fan_duty_2，將原先數值除以100。
- 由於欄位名稱和 Fan_duty_alert 相像，差異在於”duty_”後直接接數字，使用”(duty_)[1–9]”，也可以用”\d”取代”[1–9]”
- “duty”前有多個字元，使用 “.+”， 代表任意字元有1個或多個

**4.風扇負載率警示：’^(Fan).+(alert)’**

符合的欄位為Fan_duty_alert_1和Fan_duty_alert_2，分別將1和0轉換成”Yes” 和 “No”。
- “^(Fan)” 代表Fan開頭
[此項目在本例中非必要，若同時有其他包含”alert”的欄位名稱，如：Speed_alert，則為必要]
- (alert)代表包含alert
- “Fan”和”alert”中間有多個字元，使用 “.+”， 代表任意字元有1個或多個

欲進行的轉換統整如下：
1. 兩種電流 (Current、Sensor_current)除以1000。
2. 溫度和風扇負載率除以100。
3. 風扇負載率警示轉成文字，利用 applymap method 和 lambda function 將 data frame中的每個元素進行轉換，分別將 1 和 0 轉換成 “Yes” 和 “No”。

In [3]:
pattern_list = [ '.*[Cc]urrent', 'Temperature', '.+(duty_)[1-9]' , '^(Fan).+(alert)' ]

### Step 3: 透過正規表示式和迴圈一次轉換多個資料欄位

In [4]:
for k in pattern_list:
    # 找出符合pattern的資料欄位
    col = df.columns[ df.columns.str.match( pat = k) ]
    # 兩種電流 (Current、Sensor_current)除以1000 
    if k == '.*[Cc]urrent':
        df.loc[:, col] = df.loc[:, col]/1000
    # 溫度和風扇負載率除以100
    elif k in ['Temperature', '.+(duty_)[1-9]']: 
        df.loc[:, col] = df.loc[:, col]/100
    # 風扇負載率警示轉成文字  
    else:
        df.loc[:, col] = df.loc[:, col].applymap( lambda x: 'Yes' if x ==1 else 'No' )

In [5]:
df

,Serial_number,Record_time,Current_1,Current_2,Sensor_current_1,Sensor_current_2,Temperature_1,Temperature_2,Fan_duty_1,Fan_duty_2,Fan_duty_alert_1,Fan_duty_alert_2
0,A503,2021-05-24 09:00:06,3.100,3.046,3.028,3.100,29.70,27.80,0.60,0.74,No,No
1,A503,2021-05-24 09:10:06,3.716,3.662,3.644,3.734,43.50,41.88,0.60,0.74,No,No
2,A503,2021-05-24 09:20:06,3.716,3.662,3.644,3.734,43.50,41.88,0.58,0.72,No,No
3,A503,2021-05-24 09:30:07,3.189,3.136,3.118,3.189,32.12,30.34,0.56,0.70,No,No
4,A503,2021-05-24 09:40:06,3.878,3.824,3.788,3.878,45.88,44.72,0.56,0.70,No,No
5,A503,2021-05-24 09:50:07,3.878,3.824,3.788,3.878,45.88,44.72,0.58,0.72,No,No
6,A503,2021-05-24 10:00:06,3.118,3.064,3.046,3.118,29.10,27.50,0.74,0.70,No,No
7,A503,2021-05-24 10:10:06,3.752,3.716,3.680,3.770,45.13,43.00,0.77,0.70,No,No
8,A503,2021-05-24 10:20:07,3.509,3.678,3.555,3.670,44.30,42.30,0.82,0.71,Yes,No
9,A503,2021-05-24 10:30:06,3.207,3.153,3.136,3.207,28.12,26.56,0.84,0.72,Yes,No
